<a href="https://colab.research.google.com/github/Rajat-Kumar-Pandey/MACHINE-LEARNING/blob/main/sleep_detection_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf

# ------------------- Configuration -------------------
# Paths to datasets
drowsy_path = '/content/drive/MyDrive/ff_drowsy'
non_drowsy_path = '/content/drive/MyDrive/ff_non_drowsy'

# Global Settings
img_size = (64, 64)  # Target image size
padding = 10         # Padding size in pixels
batch_size = 32      # Batch size for data augmentation
input_shape = (64, 64, 1)  # Input shape for grayscale images

# ------------------- Step 1: Preprocessing -------------------
def load_and_preprocess_images_with_padding(directory, label, img_size, padding):
    """Loads and preprocesses images with padding, resizing, and normalization."""
    images = []
    labels = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img_padded = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=0)
        img_resized = cv2.resize(img_padded, img_size)
        img_normalized = img_resized / 255.0
        images.append(img_normalized)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess datasets
drowsy_images, drowsy_labels = load_and_preprocess_images_with_padding(drowsy_path, label=1, img_size=img_size, padding=padding)
non_drowsy_images, non_drowsy_labels = load_and_preprocess_images_with_padding(non_drowsy_path, label=0, img_size=img_size, padding=padding)

# Combine data
X = np.concatenate((drowsy_images, non_drowsy_images), axis=0)
y = np.concatenate((drowsy_labels, non_drowsy_labels), axis=0)

# Reshape for CNN input
X = X.reshape((-1, img_size[0], img_size[1], 1))

# ------------------- Step 2: Train-Validation-Test Split -------------------
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# One-hot encoding for categorical labels
y_train_one_hot = to_categorical(y_train, num_classes=2)
y_val_one_hot = to_categorical(y_val, num_classes=2)
y_test_one_hot = to_categorical(y_test, num_classes=2)

# ------------------- Step 3: Data Augmentation -------------------
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train_one_hot, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val_one_hot, batch_size=batch_size)

# ------------------- Step 4: Model Definition -------------------
def build_model(input_shape, learning_rate=0.001, dropout_rate=0.5, l2_lambda=0.001):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(l2_lambda), input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        Conv2D(64, (3, 3), kernel_initializer='he_normal'),
        LeakyReLU(alpha=0.1),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_lambda)),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda)),
        Dropout(dropout_rate),
        Dense(2, activation='softmax')  # Output layer
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(input_shape)

# ------------------- Step 5: Training -------------------
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    class_weight=class_weights_dict,
    callbacks=[reduce_lr]
)

# ------------------- Step 6: Evaluation -------------------
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_probs[:, 1])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


177/177 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.5041 - loss: 0.7849 - val_accuracy: 0.5037 - val_loss: 43.0082 - learning_rate: 0.0010
Epoch 2/30


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5037 - val_loss: 43.0082 - learning_rate: 0.0010
Epoch 3/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.5261 - loss: 0.6895 - val_accuracy: 0.5037 - val_loss: 41.8302 - learning_rate: 0.0010
Epoch 4/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5037 - val_loss: 41.8302 - learning_rate: 0.0010
Epoch 5/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.5153 - loss: 0.6934 - val_accuracy: 0.5037 - val_loss: 39.6063 - learning_rate: 0.0010
Epoch 6/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5037 - val_loss: 39.6063 - learning_rate: 0.0010
Epoch 7/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.5133 - loss: 0.6889 - val_accuracy: 0.5037 - val_loss: 42.1007 - learning_rate: 0.0010
Epoch 8/30

Epoch 8: ReduceLROnPlateau reducing learning rate 

In [ ]:
model.save("drowsiness_model.h5")


NameError: name 'model' is not defined